2というのが-1と1の二通りの場合に展開されるため、1 2 2 0 → 1 -1 -1 0, 1 -1 1 0, 1 1 -1 0, 1 1 1 0 と合計で四通りになります。これはx2, x3平面で見たときに第1〜4象限それぞれで別々に解を探すことになるので、“4 orthant constraints”と書いているのではないでしょうか。
「制約なし」ではなく、両方の場合で答えが欲しいときに使うんでしょうね。どういうケースで必要になるかはわかりませんが。

In [1]:
#import 
import numpy as np
import pandas as pd

In [2]:
#.matファイルの作成方法

# インクルード
import numpy as np
import scipy.io

# データ作成
data = np.arange(10)

# mat保存
#scipy.io.savemat("test.mat", {'name':data})

In [3]:
ost46 = pd.read_csv("ost46.csv", sep=',', index_col=0)
ost46

,Y,LI,SEX,AOP
0,0,1,0,1
1,0,1,0,1
2,0,1,1,0
3,0,1,1,0
4,0,1,1,0
5,0,1,1,0
6,0,1,1,1
7,0,1,1,1
8,0,1,1,1
9,0,1,1,1


In [4]:
t0 = sum(ost46['Y'])

In [5]:
t1 = np.dot(ost46['Y'], ost46['LI']) #変化する
t2 = np.dot(ost46['Y'], ost46['SEX'])
t3 = np.dot(ost46['Y'], ost46['AOP'])

In [6]:
t_list = [t0, t1, t2, t3]
t_list

[29, 19, 16, 12]

### make data

In [7]:
def dotXY(x, y):
    if x==0:
        return 0
    else:
        return 1

In [ ]:
# make .mat data
def make_mat(num_samples, feature1, feature2, feature3, y):
    mat_data = []
    
    for m in range(num_samples):
        all0 = [0]*num_samples
        all0[m] = 1
        mat_data.append(all0)
        
    mat_data.append([1]*num_samples) 
    mat_data.append([1]*num_samples)
    
    y_list = y.values.tolist()
    feature1_list = feature1.values.tolist()
    feature2_list = feature2.values.tolist()
    feature3_list = feature3.values.tolist()
    
    mat_data.append([dotXY(feature1_list[i], y_list[i]) for i in range(num_samples)])
    mat_data.append([dotXY(feature1_list[i], y_list[i]) for i in range(num_samples)])

    mat_data.append([dotXY(feature2_list[i], y_list[i]) for i in range(num_samples)])
    mat_data.append([dotXY(feature2_list[i], y_list[i]) for i in range(num_samples)])

    mat_data.append([dotXY(feature3_list[i], y_list[i]) for i in range(num_samples)])
    mat_data.append([dotXY(feature3_list[i], y_list[i]) for i in range(num_samples)])
    
    return mat_data

In [ ]:
mat_data = make_mat(X.shape[0], X['103'], X['86'], X['76'], y)
print(mat_data)

In [8]:
# make .mat data
mat_data = []
mat_data.append([1]*ost46.shape[0]) 
mat_data.append([1]*ost46.shape[0])

mat_data.append([dotXY(ost46['LI'][i], ost46['Y'][i]) for i in range(ost46.shape[0])])
mat_data.append([dotXY(ost46['LI'][i], ost46['Y'][i]) for i in range(ost46.shape[0])])

mat_data.append([dotXY(ost46['SEX'][i], ost46['Y'][i]) for i in range(ost46.shape[0])])
mat_data.append([dotXY(ost46['SEX'][i], ost46['Y'][i]) for i in range(ost46.shape[0])])

mat_data.append([dotXY(ost46['AOP'][i], ost46['Y'][i]) for i in range(ost46.shape[0])])
mat_data.append([dotXY(ost46['AOP'][i], ost46['Y'][i]) for i in range(ost46.shape[0])])

mat_shape =  [2*len(t_list), ost46.shape[0]]

In [9]:
# make .rel data
rel_data = [[">", "<"]*len(t_list)]

rel_shape = [1, 2*len(t_list)]

In [10]:
# make  .rhs data
rhs_data = [[t0]*2+[t1]*2+[t2]*2+[t3]*2]
rhs_data

rhs_shape = [1, 2*len(t_list)]

In [11]:
# make .sign data
sign_data = [[1]*ost46.shape[0]]
sign_shape = [1, ost46.shape[0]]

### make file

In [12]:
def save_system(path, shape, data):
  """
  使用例
  save_system("hoge.csv", (2, 4), [[1,1,1,1], [1,2,3,4]])
  save_system("hoge.rel", (1, 2), [["<", "<"]])
  """
  with open(path, "w") as f:
    f.write(" ".join([str(a) for a in shape])+"\n")
    for d in data:
        f.write(" ".join([str(a) for a in d])+"\n")

In [13]:
# make .mat file
save_system("trial.mat", mat_shape, mat_data)

In [14]:
# make .rel file
save_system("trial.rel", rel_shape, rel_data)

In [15]:
# make .rhs file
save_system("trial.rhs", rhs_shape, rhs_data)

In [16]:
# make .sign file
save_system("trial.sign", sign_shape, sign_data)